In [1]:
seed = 2
import numpy as np
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)

import sys
sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append("/home/webphy/Desktop/dnn_processor/dataset/")  # just to enable `dataset`


import dataset
import numpy as np
import keras
from datetime import datetime
from keras import layers
import qkeras
from qkeras.autoqkeras.utils import print_qmodel_summary
from qkeras.autoqkeras import AutoQKeras
import time

2024-02-15 19:44:06.951832: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-15 19:44:06.975286: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-15 19:44:06.975309: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-15 19:44:06.976115: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-15 19:44:06.980297: I tensorflow/core/platform/cpu_feature_guar

In [2]:
DEFAULT_BATCH_SIZE = 128
EPOCHS = 100
DEFAULT_INPUT_SIZE = 128

train_images_rgx = "../../dataset/train_set/*/*.jpeg"
test_images_rgx = "../../dataset/test_set/*/*.jpeg"

train_dataset, test_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, test_images_rgx, DEFAULT_BATCH_SIZE, DEFAULT_INPUT_SIZE
)

2024-02-15 19:44:08.171243: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-15 19:44:08.194839: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-15 19:44:08.194979: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# load the base line model

In [3]:
EXP_ID = "baseline_model_v2"

baseline_model = keras.models.load_model(
    f"../handmade/saved_models/{EXP_ID}/best_model_t/model_clean.h5",
    compile=False,
)

## ADD A LINEAR ACTIVATION LAYER BEFORE FLATTEN
x = x_in = keras.Input(shape=(128,128,3))

x = keras.layers.Activation("linear", name="qtz_inputs")(x)

for l in baseline_model.layers[1:]:
    if isinstance(l, keras.layers.Flatten):
        x = keras.layers.Activation("linear", name="qtz_flatten_input")(x)
    
    x = l(x)

model = keras.Model(x_in, x)

model.compile(
    keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(True), metrics=["accuracy"]
)

model.save("tempmodel.h5")


model = keras.models.load_model("tempmodel.h5", compile=False)
model.compile(
    keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(True), metrics=["accuracy"]
)


score = model.evaluate(test_dataset)
print("Accuracy: {}".format(score[1]))

model.summary()
baseline_model.summary()

/home/webphy/Desktop/dnn_processor_v2/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
2024-02-15 19:44:11.553910: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


5/5 [==============================] - 2s 33ms/step - loss: 0.2590 - accuracy: 0.9139
Accuracy: 0.9139280319213867
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 qtz_inputs (Activation)     (None, 128, 128, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 4)         196       
                                                                 
 activation (Activation)     (None, 32, 32, 4)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 16)        592       
                                                                 
 activation_1 (Activation)   (None, 15, 15, 16)        0         
            


# Quantizing a Model With AutoQKeras

To quantize this model with AutoQKeras, we need to define the quantization for kernels, biases and activations; forgiving factors and quantization strategy.

Below we define which quantizers are allowed for kernel, bias, activations and linear. Linear is a proxy that we use to capture Activation("linear") to apply quantization without applying a non-linear operation. In some networks, we found that this trick may be necessary to better represent the quantization space.


In [ ]:
# These are the quantizers we'll test in the bayesian optimization
quantization_config = {
    "kernel": {
        "quantized_bits(2,0,1,alpha=1.0)": 2,
        "quantized_bits(4,0,1,alpha=1.0)": 4,
        "quantized_bits(6,0,1,alpha=1.0)": 6,
        "quantized_bits(8,0,1,alpha=1.0)": 8,
    },
    "bias": {
        "quantized_bits(2,0,1,alpha=1.0)": 2,
        "quantized_bits(4,0,1,alpha=1.0)": 4,
        "quantized_bits(6,0,1,alpha=1.0)": 6,
        "quantized_bits(8,0,1,alpha=1.0)": 8,
    },
    "activation": {
        "quantized_relu(3,1)": 3,
        "quantized_relu(4,2)": 4,
        "quantized_relu(8,2)": 8,
        "quantized_relu(8,4)": 8,        
    },
    "linear": {
        "quantized_bits(2,0,1,alpha=1.0)": 2,
        "quantized_bits(4,0,1,alpha=1.0)": 4,
        "quantized_bits(6,0,1,alpha=1.0)": 6,
        "quantized_bits(8,0,1,alpha=1.0)": 8,
    },
}

# These are the layer types we will quantize
limit = {
    "Dense": [8, 8, 8],
    "Conv2D": [8, 8, 8],
    "Activation": [8],
}

# Use this if you want to minimize the model bit size
goal_bits = {
    "type": "bits",
    "params": {
        "delta_p": 5.0,  # We tolerate up to a +5% accuracy change
        "delta_n": 5.0,  # We tolerate down to a -5% accuracy change
        "rate": 2.0,  # We want a x2 times smaller model
        "stress": 1.0,  # Force the reference model size to be smaller by setting stress<1
        "input_bits": 8,
        "output_bits": 8,
        "ref_bits": 8,
        "config": {"default": ["parameters", "activations"]},
    },
}

# Use this if you want to minimize the model energy consumption
goal_energy = {
    "type": "energy",
    "params": {
        "delta_p": 5.0,
        "delta_n": 5.0,
        "rate": 1.1,
        "stress": 1.0,
        "process": "horowitz",
        "parameters_on_memory": ["sram", "sram"],
        "activations_on_memory": ["sram", "sram"],
        "rd_wr_on_io": [False, False],
        "min_sram_size": [0, 0],
        "source_quantizers": ["fp32"],
        "reference_internal": "int8",
        "reference_accumulator": "int32",
    },
}

run_config = {    
    "goal": goal_bits,
    "quantization_config": quantization_config,
    "learning_rate_optimizer": False,
    "transfer_weights": True,  # Randomely initialize weights
    "mode": "bayesian",  # This can be bayesian,random,hyperband
    "seed": 42,
    "limit": limit,
    "tune_filters": "none",
    "tune_filters_exceptions": "^dense|^conv2d_2",
    "distribution_strategy": None,
    "max_trials": 200,  # Let's just do 5 trials for this demonstrator, ideally you should do as many as possible
}


In [ ]:
import qkeras
from qkeras.autoqkeras.utils import print_qmodel_summary
from qkeras.autoqkeras import AutoQKeras
import time

In [ ]:
autoqk = AutoQKeras(
    model, output_dir="autoqqatwqkeras", metrics=["acc"], custom_objects={}, **run_config
)
autoqk.fit(train_dataset, validation_data=test_dataset, epochs=30)

In [ ]:
aqmodel = autoqk.get_best_model()
aqmodel.save_weights("autoqqatwqkeras_v2/bestmodel/model_weights.h5")
print_qmodel_summary(aqmodel)

In [ ]:
layers = [l for l in aqmodel.layers]
x = layers[0].output
for i in range(1, len(layers)):
    x = layers[i](x)

new_model = keras.Model(inputs=[layers[0].input], outputs=[x])
LOSS = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=0.0005)

new_model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=["accuracy"])
new_model.summary()
new_model.load_weights("autoqqatwqkeras_v2/bestmodel/model_weights.h5")
new_model.save("autoqqatwqkeras_v2/bestmodel/model.h5")
print_qmodel_summary(new_model)
from qkeras.estimate import print_qstats
print_qstats(new_model)

In [7]:
new_model = qkeras.utils.load_qmodel("autoqqatwqkeras_v2/bestmodel/model.h5")

In [8]:
model.evaluate(test_dataset)
new_model.evaluate(test_dataset)

1/5 [=====>........................] - ETA: 0s - loss: 0.2005 - accuracy: 0.9297

5/5 [==============================] - 0s 7ms/step - loss: 0.5603 - accuracy: 0.8044


[0.560300886631012, 0.8043818473815918]

In [9]:
new_model.fit(train_dataset, batch_size=256, validation_data=test_dataset, epochs=10)
new_model.evaluate(test_dataset)

Epoch 1/10
45/45 [==============================] - 1s 9ms/step - loss: 0.3060 - accuracy: 0.8918 - val_loss: 0.2913 - val_accuracy: 0.9061
Epoch 2/10
45/45 [==============================] - 0s 10ms/step - loss: 0.2547 - accuracy: 0.9085 - val_loss: 0.2924 - val_accuracy: 0.9061
Epoch 3/10
45/45 [==============================] - 0s 10ms/step - loss: 0.2465 - accuracy: 0.9177 - val_loss: 0.3014 - val_accuracy: 0.9030
Epoch 4/10
45/45 [==============================] - 0s 10ms/step - loss: 0.2590 - accuracy: 0.9120 - val_loss: 0.3178 - val_accuracy: 0.8998
Epoch 5/10
45/45 [==============================] - 0s 10ms/step - loss: 0.2629 - accuracy: 0.9075 - val_loss: 0.3251 - val_accuracy: 0.8826
Epoch 6/10
45/45 [==============================] - 0s 10ms/step - loss: 0.2499 - accuracy: 0.9193 - val_loss: 0.3067 - val_accuracy: 0.9030
Epoch 7/10
45/45 [==============================] - 0s 10ms/step - loss: 0.2383 - accuracy: 0.9201 - val_loss: 0.2771 - val_accuracy: 0.9171
Epoch 8/10
45/

[0.28735166788101196, 0.9045383334159851]

In [10]:
new_model.save("autoqqatwqkeras_v2/bestmodel/new_model.h5")

/home/webphy/Desktop/dnn_processor_v2/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/home/webphy/Desktop/dnn_processor_v2/.venv/lib/python3.10/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


# check metrics

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

x_test = []
y_test = []
for b in test_dataset.as_numpy_iterator():
    x_test.append(b[0])
    y_test.append(b[1])

x_test = np.concatenate(x_test, 0)
y_test = np.concatenate(y_test, 0)

y_test = np.argmax(y_test, axis=1)
y_qpred = new_model.predict(x_test)
y_qpred = np.argmax(y_qpred, axis=1)

print("accuracy_score", accuracy_score(y_test, y_qpred))
print("precision_score", precision_score(y_test, y_qpred, average="macro"))
print("recall_score", recall_score(y_test, y_qpred, average="macro"))
print("f1_score", f1_score(y_test, y_qpred, average="macro"))


20/20 [==============================] - 0s 9ms/step
accuracy_score 0.9045383411580594
precision_score 0.9045575048172492
recall_score 0.9035958373200472
f1_score 0.9036493303474245
